In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time


class ParkingLotDisplay:
    def __init__(self, surveillance_system, initial_time, playback_interval=1):
        self.surveillance_system = surveillance_system
        self.playback_interval = playback_interval
        self.current_time = initial_time
        self.playing = False
        self.thread = None

        # UI Elements
        self.time_slider = widgets.IntSlider(
            value=initial_time,
            min=0,
            max=surveillance_system.total_seconds,
            step=1,
            description="Time (s)",
            continuous_update=False
        )
        self.play_button = widgets.Button(description="Play")
        self.pause_button = widgets.Button(description="Pause")
        self.image_widget = widgets.Image(format='jpeg')
        self.parking_status_output = widgets.Output()

        # Event Handlers
        self.time_slider.observe(self.update_frame, names='value')
        self.play_button.on_click(self.start_playback)
        self.pause_button.on_click(self.stop_playback)

        # Display UI
        display(widgets.VBox([
            widgets.HBox([self.time_slider, self.play_button, self.pause_button]),
            self.image_widget,
            self.parking_status_output
        ]))
        self.update_frame(None)

    def update_frame(self, change):
        """
        Updates the frame and parking status for the current time.
        """
        try:
            # Use self.current_time to handle playback updates explicitly
            time_in_seconds = self.current_time
            frame, detected_boxes = self.surveillance_system.process_frame(time_in_seconds)

            parking_status = self.surveillance_system.get_parking_status(detected_boxes)

            # Annotate frame
            for parking_box in self.surveillance_system.parking_boxes:
                x1 = int(parking_box["x1"])
                y1 = int(parking_box["y1"])
                x2 = int(parking_box["x2"])
                y2 = int(parking_box["y2"])
                cv2.rectangle(frame, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
                cv2.putText(frame, f'Row: {parking_box["row"]}, Col: {parking_box["column"]}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

            for x1, y1, x2, y2 in detected_boxes:
                cv2.rectangle(frame, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
                cv2.putText(frame, 'Car', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            # Update parking status output
            with self.parking_status_output:
                clear_output(wait=True)  # Clear previous output
                print(parking_status)   # Print the current parking status

            # Update frame display
            ret, buffer = cv2.imencode('.jpg', frame)
            if ret:
                self.image_widget.value = buffer.tobytes()

        except Exception as e:
            print(f"Error: {e}")

    def start_playback(self, b):
        """
        Starts automatic playback by incrementing the slider value periodically.
        """
        if self.playing:
            return  # Avoid starting multiple threads
        self.playing = True
        self.thread = threading.Thread(target=self._playback_loop, daemon=True)
        self.thread.start()

    def stop_playback(self, b):
        """
        Stops the automatic playback.
        """
        self.playing = False
        if self.thread is not None:
            self.thread.join()

    def _playback_loop(self):
        """
        A loop that updates the slider and UI periodically for playback.
        """
        while self.playing and self.current_time < self.surveillance_system.total_seconds:
            time.sleep(self.playback_interval)
            self.current_time += 1
            if self.current_time > self.surveillance_system.total_seconds:
                self.current_time = self.surveillance_system.total_seconds
                break

            # Update the slider value and refresh the frame
            self.time_slider.value = self.current_time  # Update slider (visual feedback)
            self.update_frame(None)  # Explicitly refresh the frame and text output


In [5]:
from ParkingLotSurveillance import ParkingLotSurveillance

video_path = 'Parkng_Lot_Surveillance_Video.mp4'
json_path = 'flipped_car_coordinates_with_parking_positions.json'
initial_time = 280

surveillance = ParkingLotSurveillance(video_path)
surveillance.set_parking_boxes(json_path)

# Initialize the display with playback
display_ui = ParkingLotDisplay(surveillance, initial_time)

# Optional: Start playback automatically
display_ui.start_playback(None)


Using device: cuda


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ZeroDivisionError: float division by zero